# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Scraping wiki table to get the first DataFrame

### import libraries in order to scrape the table

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('libraries imported')

libraries imported


### read HTML as list and convert it into DF

In [2]:
raw_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
raw_df = pd.DataFrame(raw_list[0],columns =['Postal Code','Borough','Neighborhood'])


### drop not assigned boroughs

In [3]:
can_df = raw_df.loc[raw_df['Borough'] != 'Not assigned'].reset_index(drop=True)

### check the dataframe through "head" and "shape" methods

In [4]:
print('This is what the dataframe looks like now:')
can_df.head()



This is what the dataframe looks like now:


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
print('This is the shape of the Toronto Dataframe: ',can_df.shape)



This is the shape of the Toronto Dataframe:  (103, 3)


## Get Latitude and Logitude for each of the Postal Codes

### I found this method researching for a while
#### credits -> https://amaral.northwestern.edu/blog/getting-long-lat-list-cities

In [6]:
from opencage.geocoder import OpenCageGeocode
key = 'b15e50d653d74a1c81ada85bacc6f5f9'  # get api key from:  https://opencagedata.com
geocoder = OpenCageGeocode(key)


In [7]:
list_lat = []   # create empty lists

list_long = []

In [8]:
# testing....we need to append ', Canada' to the postal code.
query = 'M6A, Canada'  
	
results = geocoder.geocode(query)
	
print (results)

[{'annotations': {'DMS': {'lat': "43° 43' 20.28000'' N", 'lng': "79° 27' 1.44000'' W"}, 'MGRS': '17TPJ2481642196', 'Maidenhead': 'FN03gr53wi', 'Mercator': {'x': -8844378.071, 'y': 5393024.958}, 'OSM': {'note_url': 'https://www.openstreetmap.org/note/new#map=17/43.72230/-79.45040&layers=N', 'url': 'https://www.openstreetmap.org/?mlat=43.72230&mlon=-79.45040#map=17/43.72230/-79.45040'}, 'UN_M49': {'regions': {'AMERICAS': '019', 'CA': '124', 'NORTHERN_AMERICA': '021', 'WORLD': '001'}, 'statistical_groupings': ['MEDC']}, 'callingcode': 1, 'currency': {'alternate_symbols': ['C$', 'CAD$'], 'decimal_mark': '.', 'disambiguate_symbol': 'C$', 'html_entity': '$', 'iso_code': 'CAD', 'iso_numeric': '124', 'name': 'Canadian Dollar', 'smallest_denomination': 5, 'subunit': 'Cent', 'subunit_to_unit': 100, 'symbol': '$', 'symbol_first': 1, 'thousands_separator': ','}, 'flag': '🇨🇦', 'geohash': 'dpz88p3uz7ryt5c41mqw', 'qibla': 54.53, 'roadinfo': {'drive_on': 'right', 'speed_in': 'km/h'}, 'sun': {'rise': {

In [9]:
#looping through

for index, row in can_df.iterrows(): # iterate over rows in dataframe

    PostalCode = row['Postal Code']
    Borough = row['Borough']     
    Neighborhood = row['Neighborhood']
    query = str(PostalCode)+', Canada'

    results = geocoder.geocode(query)   
    lat = results[0]['geometry']['lat']
    long = results[0]['geometry']['lng']

    list_lat.append(lat)
    list_long.append(long)

	
# create new columns from lists    

can_df['lat'] = list_lat   

can_df['lon'] = list_long

In [10]:
### this is the Dataframe with lat and lon data
can_df.head()

,Postal Code,Borough,Neighborhood,lat,lon
0,M3A,North York,Parkwoods,36.433833,28.233327
1,M4A,North York,Victoria Village,43.727600,-79.314800
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655500,-79.362600
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722300,-79.450400
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",36.439174,28.233599


# Explore and cluster the neighborhoods in Toronto

In [11]:
### create a sub data frame with just Toronto data

tor_df = can_df.loc[can_df['Borough'].str.contains("Toronto")].reset_index(drop=True)
tor_df.head()



,Postal Code,Borough,Neighborhood,lat,lon
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655500,-79.362600
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",36.439174,28.233599
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657200,-79.378300
3,M5C,Downtown Toronto,St. James Town,43.651300,-79.375600
4,M4E,East Toronto,The Beaches,43.678400,-79.294100


### import libraries

In [12]:
import requests # library to handle requests
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

Solving environment: - 
  - anaconda/osx-64::ca-certificates-2020.1.1-0, anaconda/osx-64::openssl-1.1.1d-h1de35cc_4
  - anaconda/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::openssl-1.1.1d-h1de35cc_4
  - anaconda/osx-64::openssl-1.1.1d-h1de35cc_4, defaults/osx-64::ca-certificates-2020.1.1-0
  - defaults/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::openssl-1.1.1d-h1de35ccdone

# All requested packages already installed.

Solving environment: | 
  - anaconda/osx-64::ca-certificates-2020.1.1-0, anaconda/osx-64::openssl-1.1.1d-h1de35cc_4
  - anaconda/osx-64::openssl-1.1.1d-h1de35cc_4, defaults/osx-64::ca-certificates-2020.1.1-0
  - anaconda/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::openssl-1.1.1d-h1de35cc_4
  - defaults/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::openssl-1.1.1d-h1de35ccdone

# All requested packages already installed.

Folium installed
Libraries imported.


### Call Foursquare API

In [13]:
CLIENT_ID = 'PP0TFYWOO3SO353TXJVQWAQYAUVT4X5BRII4GLEM5OOLTUQ2' # your Foursquare ID
CLIENT_SECRET = 'WP1DNQ2T5JTX4UGB20FIAHSELKA3BLTDN5GZE34M43NF5JBY' # your Foursquare Secret
VERSION = '20200706'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PP0TFYWOO3SO353TXJVQWAQYAUVT4X5BRII4GLEM5OOLTUQ2
CLIENT_SECRET:WP1DNQ2T5JTX4UGB20FIAHSELKA3BLTDN5GZE34M43NF5JBY


In [14]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))




The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Clustering

In [21]:
# load libraries

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs
from urllib import request
import json # library to handle JSON files
import numpy as np
print('Libraries imported.')

Libraries imported.


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [22]:
#get lat and lon for the first neighborhood in Toronto df
neighborhood_latitude = tor_df.loc[0, 'lat'] # neighborhood latitude value
neighborhood_longitude = tor_df.loc[0, 'lon'] # neighborhood longitude value

neighborhood_name = tor_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6555, -79.3626.


In [23]:
## call foursquare

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius



url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

22 venues were returned by Foursquare.


In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=tor_df['Neighborhood'],
                                   latitudes=tor_df['lat'],
                                   longitudes=tor_df['lon']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [25]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fish Taverna,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Knitting Store,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778,0.013889,0.027778,0.000,0.00,0.013889,0.00000,0.013889,0.000000,0.000000,0.000000,0.069444,0.000000,0.000000,0.000000,0.000000,0.013889,0.013889,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.013889,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.152778,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.013889,0.00000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.013889,0.013889,0.000000,0.013889,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.013889,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,

In [27]:
### function for most common venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Boat or Ferry,Hotel,Restaurant,Pizza Place,Sporting Goods Shop,Sports Bar,Italian Restaurant,Japanese Restaurant,Bar
1,"Brockton, Parkdale Village, Exhibition Place",Tibetan Restaurant,Bar,Indian Restaurant,Pharmacy,Pizza Place,Café,Beach Bar,Boutique,Liquor Store,Fast Food Restaurant
2,"Business reply mail Processing Centre, South C...",Restaurant,Coffee Shop,Yoga Studio,Breakfast Spot,Bookstore,Sushi Restaurant,Japanese Restaurant,Italian Restaurant,Bank,Department Store
3,"CN Tower, King and Spadina, Railway Lands, Har...",Hotel,Coffee Shop,Restaurant,Italian Restaurant,Café,Pizza Place,Gym,Sushi Restaurant,Thai Restaurant,Beer Bar
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Pizza Place,Café,Bubble Tea Shop,Clothing Store,Japanese Restaurant,Middle Eastern Restaurant,Poke Place


### Run *k*-means to cluster the neighborhood into 5 clusters.

In [45]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 4, 4, 1, 4, 4, 4, 8], dtype=int32)

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = tor_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,lat,lon,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655500,-79.362600,0,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Wine Shop,Italian Restaurant,Food Truck,Event Space,Electronics Store,Distribution Center
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",36.439174,28.233599,0,Hotel,Historic Site,Greek Restaurant,Café,Gym / Fitness Center,Meze Restaurant,Fish Taverna,Grilled Meat Restaurant,Amphitheater,Arts & Crafts Store
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657200,-79.378300,0,Coffee Shop,Clothing Store,Hotel,Café,Japanese Restaurant,Cosmetics Shop,Bookstore,Diner,Fast Food Restaurant,Tea Room
3,M5C,Downtown Toronto,St. James Town,43.651300,-79.375600,0,Café,Coffee Shop,Seafood Restaurant,Restaurant,Cocktail Bar,American Restaurant,Cosmetics Shop,Italian Restaurant,Hotel,Japanese Restaurant
4,M4E,East Toronto,The Beaches,43.678400,-79.294100,0,Pub,Health Food Store,Bakery,Gastropub,Trail,Cheese Shop,Women's Store,Donut Shop,Dumpling Restaurant,Eastern European Restaurant


In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['lat'], toronto_merged['lon'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters